## Enable Amazon SageMaker Model Monitor for XGBOOST

## Step 1: Setup

In [1]:
import os
import boto3
import re
import json
from sagemaker import get_execution_role, session

region= boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket =  session.Session(boto3.Session()).default_bucket()
print("KNN: {}".format(bucket))
prefix = 'sagemakerbankfraud-KNN'
prefixtest = 'sagemakerbankfraud-KNN/test'
prefixmodel = 'sagemakerbankfraud-KNN/output/knn-210108-1950-005-ea17314e/output/'


data_capture_prefix = '{}/datacapture'.format(prefix)
s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)
reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)
code_prefix = '{}/code'.format(prefix)
#s3_code_preprocessor_uri = 's3://{}/{}/{}'.format(bucket,code_prefix, 'preprocessor.py')
#s3_code_postprocessor_uri = 's3://{}/{}/{}'.format(bucket,code_prefix, 'postprocessor.py')

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))
#print("Preproc Code path: {}".format(s3_code_preprocessor_uri))
#print("Postproc Code path: {}".format(s3_code_postprocessor_uri))



endpoint_name = 'KNN-bankfraud2021-01-08-21-11-46'

RoleArn: arn:aws:iam::246778806733:role/service-role/AmazonSageMaker-ExecutionRole-20201111T111846
KNN: sagemaker-us-east-1-246778806733
Capture path: s3://sagemaker-us-east-1-246778806733/sagemakerbankfraud-KNN/datacapture
Report path: s3://sagemaker-us-east-1-246778806733/sagemakerbankfraud-KNN/reports


## PART A: Capturing real-time inference data from Amazon SageMaker endpoints

### Upload the pre-trained model to Amazon S3

In [2]:
import boto3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefixmodel, 'model.tar.gz')).download_file('model.tar.gz')

model_file = open("model.tar.gz", 'rb')
s3_key = os.path.join(prefix, 'KNN-fraud-prediction-model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

In [3]:
from time import gmtime, strftime
from sagemaker.model import Model
#from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.image_uris import retrieve

model_name = "KNN-bank-fraud-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://{}.s3-{}.amazonaws.com/{}/KNN-fraud-prediction-model.tar.gz'.format(bucket, region, prefix)
#image_uri = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-2')
container = retrieve(framework='knn',region=boto3.Session().region_name, version='1.2-1')



model = Model(image_uri=container, model_data=model_url, role=role)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.2-1.


In [4]:
model_name

'KNN-bank-fraud-monitor-2021-01-13-15-22-42'

In [5]:
model_url

'https://sagemaker-us-east-1-246778806733.s3-us-east-1.amazonaws.com/sagemakerbankfraud-KNN/KNN-fraud-prediction-model.tar.gz'

In [6]:
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = 'KNN-bank-fraud-monitor-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=50,
                        destination_s3_uri=s3_capture_upload_path)

predictor = model.deploy(initial_instance_count=1,
                instance_type='ml.m5.xlarge',
                endpoint_name=endpoint_name,
                data_capture_config=data_capture_config)

EndpointName=KNN-bank-fraud-monitor-2021-01-13-15-22-42
---------------!

### Invoke the deployed model

In [7]:

#from sagemaker.predictor import RealTimePredictor
from sagemaker.predictor import Predictor
import time

#predictor = RealTimePredictor(endpoint=endpoint_name,content_type='text/csv')
predictor = Predictor(endpoint_name = endpoint_name)
from sagemaker.serializers import CSVSerializer
predictor.serializer = CSVSerializer()


# get a subset of test data for a quick test

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

with open('test_data_tail.csv', 'r') as f:
    for row in f:
        print(".", sep=',', end='', flush=True)
        payload = row.rstrip('\n')
        response = predictor.predict(data=payload)
        time.sleep(0.5)
        
print("Done!")

Sending test traffic to the endpoint KNN-bank-fraud-monitor-2021-01-13-15-22-42. 
Please wait...
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

### View captured data

In [8]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
sagemakerbankfraud-KNN/datacapture/KNN-bank-fraud-monitor-2021-01-13-15-22-42/AllTraffic/2021/01/13/15/30-15-041-73da7cbb-fb69-4975-9893-e9da22ddfaaa.jsonl
 sagemakerbankfraud-KNN/datacapture/KNN-bank-fraud-monitor-2021-01-13-15-22-42/AllTraffic/2021/01/13/15/31-15-450-4da9c5a2-71c7-4c1b-996e-3586e315af8a.jsonl
 sagemakerbankfraud-KNN/datacapture/KNN-bank-fraud-monitor-2021-01-13-15-22-42/AllTraffic/2021/01/13/15/32-15-514-0c376948-1b7c-425c-a766-e5e94634581d.jsonl
 sagemakerbankfraud-KNN/datacapture/KNN-bank-fraud-monitor-2021-01-13-15-22-42/AllTraffic/2021/01/13/15/33-16-134-df1dfd83-1cd8-4ced-9fe9-99a2ea85772b.jsonl
 sagemakerbankfraud-KNN/datacapture/KNN-bank-fraud-monitor-2021-01-13-15-22-42/AllTraffic/2021/01/13/15/34-16-750-683db037-50ae-46c8-829f-2f253e639621.jsonl
 sagemakerbankfraud-KNN/datacapture/KNN-bank-fraud-monitor-2021-01-13-15-22-42/AllTraffic/2021/01/13/15/35-16-824-f2f22701-cc89-4f27-a828-5e3eb1511554.jsonl
 sagemakerbankfraud-KNN/datacapture/KN

In [9]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0.23854447439353105,0.0013954830335620695,0.0003745738840052413,0.0,7.268074480395378e-05,0.0004348418034581606,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.9988459455486204,0.11643378357284305","encoding":"CSV"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"{\"predictions\": [{\"predicted_label\": 0.0}]}","encoding":"JSON"}},"eventMetadata":{"eventId":"e55413e2-f5ce-4fe0-b3f9-6ec689807ed2","inferenceTime":"2021-01-13T15:37:19Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0.2169811320754717,0.0005080713668668832,0.0,0.0,0.00013846199418233914,0.00027026723815009294,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.999491928525016,0.11643378368927686","encoding":"CSV"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"{\"predictions\": [{\"predicted_label\": 0.0}]}","encoding":"JSON"}},"eventMetadata

In [10]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "0.23854447439353105,0.0013954830335620695,0.0003745738840052413,0.0,7.268074480395378e-05,0.0004348418034581606,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.9988459455486204,0.11643378357284305",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "application/json",
      "mode": "OUTPUT",
      "data": "{\"predictions\": [{\"predicted_label\": 0.0}]}",
      "encoding": "JSON"
    }
  },
  "eventMetadata": {
    "eventId": "e55413e2-f5ce-4fe0-b3f9-6ec689807ed2",
    "inferenceTime": "2021-01-13T15:37:19Z"
  },
  "eventVersion": "0"
}


## PART B: Model Monitor - Baseling and continuous monitoring

In [11]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))





Baseline data uri: s3://sagemaker-us-east-1-246778806733/sagemakerbankfraud-KNN/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-246778806733/sagemakerbankfraud-KNN/baselining/results


In [17]:
training_data_file = open("train_with_header.csv", 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'train_with_header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)



In [18]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_knn_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_knn_monitor_baseline = my_knn_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri+'/train_with_header.csv',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)


Job Name:  baseline-suggestion-job-2021-01-13-17-04-38-032
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-246778806733/sagemakerbankfraud-KNN/baselining/data/train_with_header.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-246778806733/sagemakerbankfraud-KNN/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..........................2021-01-13 17:08:44,348 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:246778806733:processing-job/baseline-suggestion-job-2021-01-13-17-04-38-032', 'ProcessingJobName': 'baseline-suggestion-job-2021-01-13-17-04-38-032', 'Environment': {'dataset_format

2021-01-13 17:09:05,095 - DefaultDataAnalyzer - INFO - Running command: bin/spark-submit --master yarn --deploy-mode client --conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider --conf spark.serializer=org.apache.spark.serializer.KryoSerializer /opt/amazon/sagemaker-data-analyzer-1.0-jar-with-dependencies.jar --analytics_input /tmp/spark_job_config.json
2021-01-13 17:09:06 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-01-13 17:09:06 INFO  Main:28 - Start analyzing with args: --analytics_input /tmp/spark_job_config.json
2021-01-13 17:09:06 INFO  Main:31 - Analytics input path: DataAnalyzerParams(/tmp/spark_job_config.json,yarn)
2021-01-13 17:09:06 INFO  FileUtil:66 - Read file from path /tmp/spark_job_config.json.
2021-01-13 17:09:06 INFO  SparkContext:54 - Running Spark version 2.3.1
2021-01-13 17:09:06 INFO  SparkContext:54 - Submitted applicatio

2021-01-13 17:09:37 INFO  YarnClientSchedulerBackend:54 - SchedulerBackend is ready for scheduling beginning after waiting maxRegisteredResourcesWaitingTime: 30000(ms)
2021-01-13 17:09:37 WARN  SparkContext:66 - Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '/tmp' appears to be on the local filesystem.
2021-01-13 17:09:37 INFO  DatasetReader:91 - Files to process:List(file:///opt/ml/processing/input/baseline_dataset_input/train_with_header.csv)
2021-01-13 17:09:37 INFO  SharedState:54 - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/usr/spark-2.3.1/spark-warehouse').
2021-01-13 17:09:37 INFO  SharedState:54 - Warehouse path is 'file:/usr/spark-2.3.1/spark-warehouse'.
2021-01-13 17:09:37 INFO  StateStoreCoordinatorRef:54 - Registered StateStoreCoordinator endpoint
2021-01-13 17:09:38 INFO  FileSourceStrategy:54 - Pruning directories with: 
2021-01-13 17:09:38 INFO  File

2021-01-13 17:09:53 INFO  BlockManagerInfo:54 - Added rdd_11_0 in memory on algo-1:32815 (size: 102.7 MB, free: 5.7 GB)
2021-01-13 17:09:53 INFO  TaskSetManager:54 - Finished task 0.0 in stage 1.0 (TID 1) in 9884 ms on algo-1 (executor 1) (1/1)
2021-01-13 17:09:53 INFO  YarnScheduler:54 - Removed TaskSet 1.0, whose tasks have all completed, from pool 
2021-01-13 17:09:53 INFO  DAGScheduler:54 - ResultStage 1 (head at DataAnalyzer.scala:79) finished in 9.930 s
2021-01-13 17:09:53 INFO  DAGScheduler:54 - Job 1 finished: head at DataAnalyzer.scala:79, took 9.940422 s
2021-01-13 17:09:53 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
2021-01-13 17:09:53 INFO  CodeGenerator:54 - Code generated in 32.790241 ms
2021-01-13 17:09:54 INFO  SparkContext:54 - Starting job: collect at AnalysisRunner.scala:313
2021-01-13 17:09:54 INFO  DAGScheduler:54 - Registering RDD 2

2021-01-13 17:11:28 INFO  TaskSetManager:54 - Starting task 3.0 in stage 4.0 (TID 12, algo-1, executor 1, partition 3, PROCESS_LOCAL, 8341 bytes)
2021-01-13 17:11:28 INFO  TaskSetManager:54 - Finished task 2.0 in stage 4.0 (TID 11) in 45490 ms on algo-1 (executor 1) (1/6)
2021-01-13 17:11:31 INFO  TaskSetManager:54 - Starting task 4.0 in stage 4.0 (TID 13, algo-1, executor 1, partition 4, PROCESS_LOCAL, 8341 bytes)
2021-01-13 17:11:31 INFO  TaskSetManager:54 - Finished task 0.0 in stage 4.0 (TID 9) in 48501 ms on algo-1 (executor 1) (2/6)
2021-01-13 17:11:31 INFO  TaskSetManager:54 - Starting task 5.0 in stage 4.0 (TID 14, algo-1, executor 1, partition 5, PROCESS_LOCAL, 8341 bytes)
2021-01-13 17:11:31 INFO  TaskSetManager:54 - Finished task 1.0 in stage 4.0 (TID 10) in 48640 ms on algo-1 (executor 1) (3/6)
2021-01-13 17:11:56 INFO  TaskSetManager:54 - Finished task 5.0 in stage 4.0 (TID 14) in 24991 ms on algo-1 (executor 1) (4/6)
2021-01-13 17:12:09 INFO  TaskSetManager:54 - Finished 

2021-01-13 17:12:20 INFO  TaskSetManager:54 - Starting task 3.0 in stage 6.0 (TID 20, algo-1, executor 1, partition 3, PROCESS_LOCAL, 8341 bytes)
2021-01-13 17:12:20 INFO  TaskSetManager:54 - Finished task 2.0 in stage 6.0 (TID 19) in 6287 ms on algo-1 (executor 1) (1/6)
2021-01-13 17:12:23 INFO  TaskSetManager:54 - Starting task 4.0 in stage 6.0 (TID 21, algo-1, executor 1, partition 4, PROCESS_LOCAL, 8341 bytes)
2021-01-13 17:12:23 INFO  TaskSetManager:54 - Finished task 1.0 in stage 6.0 (TID 18) in 9215 ms on algo-1 (executor 1) (2/6)
2021-01-13 17:12:23 INFO  TaskSetManager:54 - Starting task 5.0 in stage 6.0 (TID 22, algo-1, executor 1, partition 5, PROCESS_LOCAL, 8341 bytes)
2021-01-13 17:12:23 INFO  TaskSetManager:54 - Finished task 0.0 in stage 6.0 (TID 17) in 9973 ms on algo-1 (executor 1) (3/6)
2021-01-13 17:12:27 INFO  TaskSetManager:54 - Finished task 5.0 in stage 6.0 (TID 22) in 3162 ms on algo-1 (executor 1) (4/6)
2021-01-13 17:12:28 INFO  TaskSetManager:54 - Finished tas

2021-01-13 17:12:38 INFO  ConstraintGenerator:50 - Constraints: {
  "version" : 0.0,
  "features" : [ {
    "name" : "isfraud",
    "inferred_type" : "Integral",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "step",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "amount",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "oldbalanceorg",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "newbalanceorig",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "oldbalancedest",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constra

### Explore the generated constraints and statistics

In [28]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
sagemakerbankfraud-KNN/baselining/results/constraints.json
 sagemakerbankfraud-KNN/baselining/results/statistics.json


In [29]:
import pandas as pd

baseline_job = my_knn_monitor.latest_baselining_job
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data,string_statistics.common.num_present,string_statistics.common.num_missing,string_statistics.distinct_count
0,isfraud,Integral,4453834.0,0.0,0.001289,5.742000e+03,0.035883,0.000,1.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[], [1.0], [1.0, 1.0], [1.0, 0.0, 0.0, 0.0, 0...",NaN,NaN,NaN
1,step,Fractional,4453834.0,0.0,0.327665,1.459366e+06,0.191831,0.001,1.001000,"[{'lower_bound': 0.001, 'upper_bound': 0.10099...",0.64,2048.0,"[[], [0.9497870619946092], [0.9497870619946092...",NaN,NaN,NaN
2,amount,Fractional,4453834.0,0.0,0.002947,1.312748e+04,0.006565,0.001,1.001000,"[{'lower_bound': 0.001, 'upper_bound': 0.10099...",0.64,2048.0,"[[], [0.13344229493225968], [0.176201804032018...",NaN,NaN,NaN
3,oldbalanceorg,Fractional,4453834.0,0.0,0.014989,6.675692e+04,0.048471,0.001,0.962924,"[{'lower_bound': 0.001, 'upper_bound': 0.09719...",0.64,2048.0,"[[], [0.45129925084197775], [0.485961622087762...",NaN,NaN,NaN
4,newbalanceorig,Fractional,4453834.0,0.0,0.018235,8.121689e+04,0.058967,0.001,0.955245,"[{'lower_bound': 0.001, 'upper_bound': 0.09642...",0.64,2048.0,"[[], [0.5469722784934782], [0.5860722781210473...",NaN,NaN,NaN
5,oldbalancedest,Fractional,4453834.0,0.0,0.004089,1.821373e+04,0.009489,0.001,1.001000,"[{'lower_bound': 0.001, 'upper_bound': 0.10099...",0.64,2048.0,"[[], [0.16084775051445327], [0.104977826011040...",NaN,NaN,NaN
6,newbalancedest,Fractional,4453834.0,0.0,0.004438,1.976584e+04,0.010267,0.001,1.001000,"[{'lower_bound': 0.001, 'upper_bound': 0.10099...",0.64,2048.0,"[[], [0.16167364680373192], [0.104436896642926...",NaN,NaN,NaN
7,morning,Fractional,4453834.0,0.0,0.348068,1.550236e+06,0.476358,0.000,1.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[], [1.0], [1.0, 1.0], [1.0, 1.0, 0.0, 0.0, 0...",NaN,NaN,NaN
8,afternoon,Fractional,4453834.0,0.0,0.323565,1.441105e+06,0.467836,0.000,1.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[], [1.0], [1.0, 1.0], [1.0, 1.0, 0.0, 0.0, 0...",NaN,NaN,NaN
9,evening,Fractional,4453834.0,0.0,0.328367,1.462493e+06,0.469619,0.000,1.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[], [1.0], [1.0, 1.0], [1.0, 1.0, 0.0, 0.0, 0...",NaN,NaN,NaN


In [30]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,isfraud,Integral,1.0,True
1,step,Fractional,1.0,True
2,amount,Fractional,1.0,True
3,oldbalanceorg,Fractional,1.0,True
4,newbalanceorig,Fractional,1.0,True
5,oldbalancedest,Fractional,1.0,True
6,newbalancedest,Fractional,1.0,True
7,morning,Fractional,1.0,True
8,afternoon,Fractional,1.0,True
9,evening,Fractional,1.0,True


## Step 3: Enable continous monitoring

In [31]:
# First, copy over some test scripts to the S3 bucket so that they can be used for pre and post processing if using
#boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/preprocessor.py").upload_file('preprocessor.py')
#boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/postprocessor.py").upload_file('postprocessor.py')

### start some artificial traffic

In [32]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_knn_name = 'KNN-bank-fraud-monitor-schedule' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
s3_report_path = baseline_results_uri
my_knn_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_knn_name,
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    statistics=my_knn_monitor.baseline_statistics(),
    constraints=my_knn_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.daily(),
    enable_cloudwatch_metrics=True,

)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
It seems that this object was already used to create an Amazon Model Monitoring Schedule. To create another, first delete the existing one using my_monitor.delete_monitoring_schedule().


ValueError: It seems that this object was already used to create an Amazon Model Monitoring Schedule. To create another, first delete the existing one using my_monitor.delete_monitoring_schedule().

In [33]:
predictor.endpoint_name

'KNN-bank-fraud-monitor-2021-01-13-15-22-42'

In [34]:
from threading import Thread
from time import sleep
import time

#endpoint_name=predictor.endpoint
endpoint_name=predictor.endpoint_name

runtime_client = boto3.client('runtime.sagemaker')

# (just repeating code from above for convenience/ able to run this section independently)
def invoke_endpoint(ep_name, file_name, runtime_client):
    with open(file_name, 'r') as f:
        for row in f:
            payload = row.rstrip('\n')
            response = runtime_client.invoke_endpoint(EndpointName=ep_name,
                                          ContentType='text/csv', 
                                          Body=payload)
            response['Body'].read()
            time.sleep(1)
            
def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, 'test_data_tail.csv', runtime_client)
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

In [35]:
desc_schedule_result = my_knn_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

Schedule status: Scheduled


### List executions

In [ ]:
mon_executions = my_knn_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_knn_monitor.list_executions()

No executions found for schedule. monitoring_schedule_name: KNN-bank-fraud-monitor-schedule2021-01-13-17-13-25
We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...
Waiting for the 1st execution to happen...


In [ ]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

In [ ]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))

In [ ]:
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

### Violations Report

In [ ]:

violations = my_knn_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', None)
constraints_df = pd.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

### Delete the resources

In [ ]:
my_knn_monitor.delete_monitoring_schedule()
time.sleep(60) # actually wait for the deletion

In [ ]:
predictor.delete_endpoint()

In [ ]:
predictor.delete_model()